In [6]:
import cv2
import numpy as np
from scipy.spatial import distance as dist
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

from keras.preprocessing import image

In [7]:
# Create the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [8]:
faceCascade = cv2.CascadeClassifier("Resources/cascade/haarcascade_frontalface_default.xml")

model2 = keras.models.clone_model(model)

model.load_weights('Resources/Trained_Data/test1.h5')
model2.load_weights('Resources/Trained_Data/test2.h5')

# dictionary which assigns each label an emotion (alphabetical order)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [9]:
img = cv2.imread("Resources/images/faces.jpg")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = faceCascade.detectMultiScale(img, 1.3, 5)

img2 = img.copy()

for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
    roi_gray=gray[y:y+w,x:x+h]
    roi_gray=cv2.resize(roi_gray,(48,48))
    img_pixels = image.img_to_array(roi_gray)
    img_pixels = np.expand_dims(img_pixels, axis = 0)
    img_pixels /= 255
    
    prediction = model.predict(img_pixels)
    maxindex = int(np.argmax(prediction))
    cv2.putText(img, emotion_dict[maxindex], (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    cv2.rectangle(img2,(x,y),(x+w,y+h),(255,0,0),thickness=7)
    prediction2 = model2.predict(img_pixels)
    maxindex2 = int(np.argmax(prediction2))
    cv2.putText(img2, emotion_dict[maxindex2], (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    #prediction, idx, probability = learn.predict(Image(pil2tensor(roi_gray, np.float32).div_(225)))
    #cv2.putText(img, str(prediction), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (225, 255, 255), 2)

resized_img = cv2.resize(img, (1000, 700))
cv2.imshow('Facial emotion analysis ',resized_img)


resized_img2 = cv2.resize(img2, (1000, 700))
cv2.imshow('Facial emotion analysis2 ',resized_img2)

cv2.waitKey(0)

-1